In [5]:
import pyfesom2 as pf
import numpy as np

In [8]:
pf.cross_section_transports('FRAMSTRAIT',
                            mesh_path='/Users/fheukamp/Desktop/fesom2_test_year/',
                            data_path='/Users/fheukamp/Desktop/fesom2_test_year/',
                            mesh_diag_path='/Users/fheukamp/Desktop/fesom2_test_year/fesom.mesh.diag.nc',
                            years=np.arange(2000,2001),
                            use_great_circle=False,
                            how='ori',
                            add_IT=True,
                            add_TS=True
                           )

/Users/fheukamp/Desktop/fesom2_test_year/pickle_mesh_py3_fesom2
The usepickle == True)
The pickle file for FESOM2 exists.
The mesh will be loaded from /Users/fheukamp/Desktop/fesom2_test_year/pickle_mesh_py3_fesom2


/Users/fheukamp/Documents/Software/pyfesom2/pyfesom2/transport.py:157: UserWarning: For zonal sections the length of the non-great-circle section is computed with a                        reference length of 111.568 km/°E * cos(lat)
  warnings.warn('For zonal sections the length of the non-great-circle section is computed with a \


[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s


NameError: name 'pf' is not defined